<a href="https://colab.research.google.com/github/iqbalmohomed/LLM4BUS/blob/main/LLM4BUS_hw6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install langchain
!pip install langchain-community
!pip install cohere

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.5/807.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 36.6 MB/s eta 0:00:00
  Attempting uninstall: importlib_metadata
    Found existing installation: importlib_metadata 7.0.2
    Uninstalling importlib_metadata-7.0.2:
      Successfully uninstalled importlib_metadata-7.0.2


# Homework 6: Use Cohere along with LangChain/FAISS to implement simple RAG over a pdf document.

In [3]:
# Get Cohere key from Colab
from google.colab import userdata
cohere_key = userdata.get('cohere_key')
# Save to an env variable
import os
os.environ["COHERE_API_KEY"] = cohere_key

In [6]:
!wget -O alphabet_10k.pdf "https://abc.xyz/assets/investor/static/pdf/20230203_alphabet_10K.pdf"

--2024-03-11 21:53:38--  https://abc.xyz/assets/investor/static/pdf/20230203_alphabet_10K.pdf
Resolving abc.xyz (abc.xyz)... 74.125.134.113, 74.125.134.138, 74.125.134.100, ...
Connecting to abc.xyz (abc.xyz)|74.125.134.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 897814 (877K) [application/pdf]
Saving to: ‘alphabet_10k.pdf’

alphabet_10k.pdf    100%[===================>] 876.77K  --.-KB/s    in 0.01s   

2024-03-11 21:53:38 (88.7 MB/s) - ‘alphabet_10k.pdf’ saved [897814/897814]



In [7]:
!pip install pypdf
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 32.5 MB/s eta 0:00:00


In [9]:
# Setup Documents to Index
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

pdf_path = "alphabet_10k.pdf"
loader = PyPDFLoader(file_path = pdf_path)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1500, chunk_overlap=50,separator="\n")
docs = text_splitter.split_documents(documents=documents)
print(len(docs))

262


In [11]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import CohereEmbeddings

embeddings = CohereEmbeddings(model="embed-english-light-v3.0")
vectorstore = FAISS.from_documents(docs,embeddings)
vectorstore.save_local("faiss_index_alphabet10k")

In [13]:
#Lets load the saved embeddings and do some RAG (Persisting the vector store and loading it again for learning purposes)
myvectorstore = FAISS.load_local("faiss_index_alphabet10k", embeddings,allow_dangerous_deserialization=True)

In [15]:
# Now, lets use Langchain to do RAG
from langchain.chains import RetrievalQA
from langchain_community.chat_models import ChatCohere

qa = RetrievalQA.from_chain_type(llm=ChatCohere(), chain_type="stuff", retriever= myvectorstore.as_retriever())

In [18]:
qa.invoke("What is the outlook for Alphabet?")

{'query': 'What is the outlook for Alphabet?',
 'result': 'As of my knowledge cutoff date of January 2023, the outlook for Alphabet Inc. (the parent company of Google) appears positive, with several analysts and investors expressing optimism about its future prospects. Alphabet has been benefiting from its robust advertising business, dominant position in the search engine market, and successful ventures into cloud computing.\n\nAccording to financial analysts, Alphabet\'s diverse business segments, including its core advertising business, cloud services, and hardware products, are expected to drive its growth in the coming years. The company\'s focus on innovation and its continued investments in research and development are seen as key factors for long-term success.\n\nHowever, there are also challenges to consider. Regulatory concerns, especially related to data privacy and antitrust issues, remain a potential headache for the company. Alphabet has faced increased scrutiny and fines

In [19]:
qa.invoke("According to the document, what is the outlook for Alphabet?")

{'query': 'According to the document, what is the outlook for Alphabet?',
 'result': "The outlook for Alphabet Inc. (the parent company of Google) is not explicitly mentioned in the provided document. However, some insights and implications can be drawn from the information presented.\n\nThe document highlights Alphabet's diverse business portfolio and its focus on innovative and growth-oriented ventures. With a strong emphasis on technology and a global presence, Alphabet appears to have a positive outlook in terms of its potential for growth and market reach.\n\nAlphabet's core business, Google, continues to dominate the search engine market and has a strong position in the digital advertising industry. The company's investment in artificial intelligence and machine learning can be beneficial in enhancing its services and developing new revenue streams.\n\nThe mention of management's commitment to disciplined investment and focus on long-term growth strategies indicates a thoughtful 

In [20]:
qa.invoke("What are the risks for Alphabet?")

{'query': 'What are the risks for Alphabet?',
 'result': "As one of the world's leading technology companies, Alphabet Inc. (formerly Google Inc.) faces a variety of risks that can potentially impact its operations and financial performance. Here are some key risks the company faces:\n\n1. Regulatory and Legal Risks: Alphabet, like other tech giants, faces increased scrutiny from regulatory bodies worldwide. Risks related to data privacy, antitrust laws, and consumer protection regulations could potentially harm the company's operations and financial health. Legal actions and regulatory investigations can result in significant fines, changes to business practices, and reputational damage.\n\n2. Competition: The technology industry is highly competitive, and Alphabet competes with innovative and established companies across its various business segments. Competition from companies like Apple, Microsoft, Amazon, and Facebook can erode Alphabet's market share and put pressure on its profi

In [21]:
qa.invoke("Conscisely describe how Alphabet makes money")

{'query': 'Conscisely describe how Alphabet makes money',
 'result': "Alphabet makes money primarily through its ownership of Google and its advertising business. Google dominates the global search engine market and uses this position to sell ads that appear in search results and on websites that use Google's advertising services. Additionally, Google's YouTube is a significant revenue stream through its advertising model. Alphabet also earns from cloud computing services, hardware sales, and other smaller businesses like digital mapping and investment arms. The company's diverse business ventures, including those in healthcare and autonomous driving, also contribute to its revenue streams. Alphabet's main revenue source is still advertising, which it places strategically using data analysis."}

In [22]:
qa.invoke("Conscisely describe how Alphabet is improving advertising")

{'query': 'Conscisely describe how Alphabet is improving advertising',
 'result': "Alphabet, Google's parent company, is improving advertising through its various business ventures. Here are some ways they are doing this:\n\n1. Google's Search and Maps Services: By providing accurate and detailed location data, Alphabet can offer targeted ads to users based on their physical proximity to businesses, enhancing local advertising.\n\n2. Google Ads: The company continues to refine its Google Ads platform, enabling businesses to create and manage targeted online ads shown across Google's extensive network. New features and improvements make advertising more effective and measurable.\n\n3. YouTube: Alphabet leverages YouTube's immense popularity to offer visual advertising options. Companies can place ads before, during, or alongside videos, reaching a vast audience. Targeting options allow ads to be directed at specific demographics or interests.\n\n4. Machine Learning and AI: Alphabet's ma